In [19]:
from spikeTrainAnalysis import *
%matplotlib inline

In [26]:
dDir="trenesBursts/aleSpikeTimes/"
stCnt= xtractTrains(dataDir=dDir,prefix="CntSt",suff=".mat")
stAnt=xtractTrains(dataDir=dDir,prefix="AntSt",suff=".mat")
stLes=xtractTrains(dataDir=dDir,prefix="LesSt",suff=".mat")
print(stCnt)

[array([  4.00000000e-03,   1.04000000e-01,   1.86500000e-01, ...,
         3.59873667e+02,   3.59938333e+02,   3.59994000e+02]), array([  2.66666667e-02,   1.18833333e-01,   2.57333333e-01, ...,
         2.00600500e+02,   2.00741167e+02,   2.00920000e+02]), array([  6.15000000e-02,   1.84666667e-01,   3.03666667e-01, ...,
         3.59609000e+02,   3.59780000e+02,   3.59973167e+02]), array([  6.15000000e-02,   1.84666667e-01,   3.03666667e-01, ...,
         3.59609000e+02,   3.59780000e+02,   3.59973167e+02]), array([  6.15000000e-02,   1.84666667e-01,   3.03666667e-01, ...,
         3.59609000e+02,   3.59780000e+02,   3.59973167e+02]), array([   0.20316667,    0.55866667,    1.69933333,    2.1825    ,
          2.95833333,    4.37683333,    5.48833333,    6.3225    ,
          8.795     ,    9.4465    ,    9.89783333,   10.81283333,
         11.26433333,   13.3185    ,   16.25916667,   18.284     ,
         18.672     ,   19.39466667,   19.877     ,   20.40466667,
         21.0906666

In [60]:
def xcorr(tRef, t2, binSize=0.01, maxTime=0.1, minTime=-0.1): 
    """ Example:
    c,b= xcorr(tRef,t2,maxTime=1.0, binSize=0.001, minTime=0)
    """
    nRef= len(tRef); n2= len(t2)
    bins= sc.arange(minTime-binSize/2,1e-8+maxTime+binSize/2,binSize)
    counts= sc.zeros(len(bins)-1)
    # For each spike in the reference cell
    for i in sc.arange(len(tRef)):
        yi = t2 -tRef[i] 
        ci,bi=sc.histogram(yi, bins)
        counts = counts + ci
    return counts/len(tRef), bins[:-1]+binSize/2

def trainBursts(train, aBinSize=0.05, maxTime=1.0, alpha=0.95):
    """
    Find bursts of high activity (high instantaneous firing rate) in a train of pulses with respect to a statistical threshold upQuant. Example:
    bursts= trainBursts(train, aBinSize=0.05, maxTime=1.0, alpha=0.95)
    """
    nSpikes=len(train)
    isi= sc.zeros(nSpikes); 
    dIFR= sc.zeros(nSpikes)
    isi[1:]= train[1:]-train[:-1]
    ifr=1/isi
    dIFR[1:]= (((ifr[1:]-ifr[:-1])/isi[1:]) + ((ifr[1:]-ifr[:-1])/isi[:-1]))/2.0
    ifrCDF,ifrCDFInverse= calcCDF(ifr,graph=0)
    #isiCDF,isiCDFInverse= calcCDF(isi,graph=0)
    # Find spikes during high activity
    ifrUpThresh= calcThresholdsFromCDF(ifrCDFInverse, (alpha,))
    ifrDnThresh= calcThresholdsFromCDF(ifrCDFInverse, (1-alpha,))
    #isiThresh= calcThresholdsFromCDF(isiCDFInverse, (alpha,))
    #rHighInds= sc.where( ifr>ifrThresh)[0]
    rHighInds= sc.where( ifr>ifrUpThresh)[0]
    lHighInds= rHighInds-1
    highInds= sc.union1d(rHighInds,lHighInds)
    highSpikeTimes= train[highInds]
    #lowSpikeTimes= train[lowInds]
    aa= sc.zeros(len(highInds))
    aa[1:]= sc.diff(highInds)
    #bb= sc.zeros(len(lowInds))
    #bb[1:]= sc.diff(lowInds)
    startInds=sc.where(aa!=1)[0]
    burstStarts= highSpikeTimes[startInds]
    burstEnds= highSpikeTimes[startInds-1]
    nBursts= len(burstStarts)
    burstStarts=burstStarts[:-1]
    burstEnds=burstEnds[1:]    
    pBurst = sc.float32(nBursts)/nSpikes
    burstDurs = burstEnds-burstStarts
    c,b= xcorr(train,train, aBinSize, maxTime, minTime=0)
    cHz = c/aBinSize
    c[0]=0.0
    bursts={"train": train, 
        "highInds":highInds, "highSpikeTimes":highSpikeTimes, 
        "burstStarts":burstStarts, "burstEnds":burstEnds, 
        "nBursts":nBursts, "nSpikes": nSpikes, "pBurst": pBurst, "burstDurs": burstDurs,
        "alpha":alpha,
        "ifr":ifr, "ifrCDF":ifrCDF, "ifrCDFInverse":ifrCDFInverse,"ifrThresh":ifrUpThresh,
        "isi":isi, #"isiCDF":isiCDF, "isiCDFInverse":isiCDFInverse,"isiThresh":isiThresh,
        "dIFR":dIFR, 
        "aCorrHz":cHz, "aCorrBins":b}
    return bursts


In [64]:
cell=0
st=stCnt[cell]
bursts=trainBursts(train=st, aBinSize=0.05, maxTime=1.0, alpha=0.95)
bursts.keys()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel-4.3.1-py3.5.egg/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in true_divide


dict_keys(['highInds', 'nSpikes', 'burstDurs', 'alpha', 'ifrThresh', 'burstStarts', 'burstEnds', 'isi', 'ifrCDFInverse', 'dIFR', 'aCorrHz', 'pBurst', 'highSpikeTimes', 'train', 'ifrCDF', 'ifr', 'nBursts', 'aCorrBins'])

In [16]:
dataDir="trenesBursts/aleSpikeTimes/"
thrAntFile="AntThreshold.mat"
aa=io.loadmat(dataDir+thrAntFile)
print(aa.keys())
print(aa["AntBurst"])
#sc.squeeze(io.loadmat(dataDir+thrAntFile)[varName][0][0])

dict_keys(['__globals__', '__header__', 'AntBurst', '__version__'])
[[ ([[array(['A6lmVm'], 
      dtype='<U6'), array(['A14klmVm'], 
      dtype='<U8'), array(['A27lmoVm'], 
      dtype='<U8'), array(['A28mnoVm'], 
      dtype='<U8'), array(['A39lVm'], 
      dtype='<U6'), array(['A43fVm'], 
      dtype='<U6'), array(['A45J'], 
      dtype='<U4'), array(['A98oVm'], 
      dtype='<U6'), array(['A71gVm'], 
      dtype='<U6'), array(['A76tVm'], 
      dtype='<U6'), array(['A69iVm'], 
      dtype='<U6'), array(['A101mVm'], 
      dtype='<U7')]], [[array([[ 0.252     ,  0.39866667,  1.2735    ,  0.26633333,  0.44833333,
         0.61833333,  0.47883333,  0.96166667,  0.2555    ,  0.28233333,
         0.23466667,  1.01366667,  1.55783333,  0.215     ,  0.17766667,
         1.37983333,  0.83766667,  0.26766667,  0.47083333,  0.296     ,
         0.2615    ,  0.57016667,  0.18033333,  0.98316667,  0.57833333,
         0.81416667,  0.62416667,  0.378     ,  1.28766667,  0.772     ,
         1.